In [1]:
# Import dependencies
import os
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import pandas as pd
import numpy as np
import tensorflow as tf
import logging
from tqdm import tqdm

# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, BinaryFocalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, NUM_CLASSES, TF_SHUFFLE_SIZE, TF_BATCH_SIZE
from src.model.tensorflow_utils import load_dataset, apply_augmentation_to_dataset, oversample_minority_classes, optimize_dataset, count_dataset_size
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_loss_monitor, setup_garbage_collector, get_metrics
from src.model.tensorflow_utils import calculate_class_weights, show_class_weights, start_or_resume_training, analyze_class_distribution
from src.model.densnet.tensorflow_dense_net_121 import build_densenet121

# Input Data
initial_epoch   = None
resume_training = False
checkpoint_path = None
model_name      = "DenseNet121_v3"

In [2]:
train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)
test_ds  = load_dataset(f"{TF_RECORD_DATASET}/test.tfrecord", TF_BUFFER_SIZE)

I0000 00:00:1742893896.683440 36020418 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1742893896.683468 36020418 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Optimize Dataset for rare clasess
class_weights = calculate_class_weights(train_ds, NUM_CLASSES)
train_ds      = oversample_minority_classes(train_ds, class_weights)
class_weights = calculate_class_weights(train_ds, NUM_CLASSES)

In [4]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Testing
train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds  = optimize_dataset(val_ds, TF_BATCH_SIZE)

Counting samples: 108109 samples [01:14, 1443.96 samples/s] 
Counting samples: 15391 samples [00:17, 879.98 samples/s]


In [5]:
# Setup Model Deps
# Setup Loggers
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
loss_monitor      = setup_loss_monitor(MODELS_PATH, model_name, logger, val_ds, resume_training=resume_training, initial_epoch=initial_epoch)
garbage_collector = setup_garbage_collector(logger)
metrics           = get_metrics()

# Setup compile arguments
focal_loss = BinaryCrossentropy(from_logits=False, label_smoothing=0.01)
reduce_lr  = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max")

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max")

2025-03-25 10:15:39 - INFO - Resuming from existing metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v2_1/train_metrics.csv
2025-03-25 10:15:39 - INFO - Resuming from existing validation metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v2_1/val_metrics.csv
2025-03-25 10:15:39 - INFO - Cleaned training metrics file, kept 84450 records before epoch 26
2025-03-25 10:15:39 - INFO - Cleaned validation metrics file, kept 25 records before epoch 26
2025-03-25 10:15:39 - INFO - Found 84450 existing training records
2025-03-25 10:15:39 - INFO - Found 25 existing validation records
2025-03-25 10:15:39 - INFO - Cleaned loss analysis metrics file, kept 375 records before epoch 26
2025-03-25 10:15:39 - INFO - Resuming from existing loss analysis file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v2_1/loss_analysis_metrics.csv


In [6]:
# Model Training
model          = build_densenet121(NUM_CLASSES, use_se=True)
compile_kwargs = {'optimizer': Adam(learning_rate=1e-4, clipnorm=1.0), 'loss': focal_loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    30,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, loss_monitor, garbage_collector], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-03-25 10:15:41 - INFO - Resuming from existing metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v2_1/train_metrics.csv
2025-03-25 10:15:41 - INFO - Resuming from existing validation metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v2_1/val_metrics.csv
2025-03-25 10:15:41 - INFO - Cleaned training metrics file, kept 84450 records before epoch 26
2025-03-25 10:15:41 - INFO - Cleaned validation metrics file, kept 25 records before epoch 26
2025-03-25 10:15:41 - INFO - Found 84450 existing training records
2025-03-25 10:15:41 - INFO - Found 25 existing validation records
2025-03-25 10:15:41 - INFO - Cleaned loss analysis metrics file, kept 375 records before epoch 26
2025-03-25 10:15:41 - INFO - Resuming from existing loss analysis file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v2_1/loss_analysis_metrics.csv


Loading full model from checkpoint: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/DenseNet121_v2_1/checkpoints/cp-0025.keras


2025-03-25 10:15:44 - INFO - 
=== Training Started ===

2025-03-25 10:15:44 - INFO - Batch Size: 32
2025-03-25 10:15:44 - INFO - Optimizer: Adam
2025-03-25 10:15:44 - INFO - 

2025-03-25 10:15:44 - INFO - 
=== Starting Epoch 26 ===



Epoch 26/30
3378/3378 ━━━━━━━━━━━━━━━━━━━━ 0s 636ms/step - accuracy: 0.9076 - auc: 0.7671 - f1_score: 0.1098 - loss: 0.1807 - precision: 0.4833 - recall: 0.0801   

2025-03-25 10:54:25 - INFO - 
=== Epoch 26 Summary ===
2025-03-25 10:54:25 - INFO - Time: 2321.45s
2025-03-25 10:54:25 - INFO - Training   - accuracy: 0.9076 - auc: 0.7670 - f1_score: 0.1071 - loss: 0.1814 - precision: 0.4812 - recall: 0.0772 - learning_rate: 0.0001
2025-03-25 10:54:25 - INFO - Validation - accuracy: 0.9128 - auc: 0.7389 - f1_score: 0.1035 - loss: 0.2643 - precision: 0.4375 - recall: 0.0687
2025-03-25 10:54:25 - INFO - ==================================================



...

2025-03-25 14:02:20 - INFO - 
=== Epoch 30 Summary ===
2025-03-25 14:02:20 - INFO - Time: 2543.81s
2025-03-25 14:02:20 - INFO - Training   - accuracy: 0.9077 - auc: 0.7759 - f1_score: 0.1205 - loss: 0.1791 - precision: 0.4858 - recall: 0.0874 - learning_rate: 0.0001
2025-03-25 14:02:20 - INFO - Validation - accuracy: 0.9087 - auc: 0.7294 - f1_score: 0.1004 - loss: 0.2727 - precision: 0.3534 - recall: 0.0810
2025-03-25 14:02:20 - INFO - ==================================================



1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

2025-03-25 14:04:09 - INFO - 
Loss Analysis - Atelectasis
2025-03-25 14:04:09 - INFO - Confidence Distribution:
2025-03-25 14:04:09 - INFO - -- High (>0.9): 0.00%
2025-03-25 14:04:09 - INFO - -- Medium (0.6-0.9): 0.81%
2025-03-25 14:04:09 - INFO - -- Uncertain (0.4-0.6): 6.44%
2025-03-25 14:04:09 - INFO - -- Low (<0.4): 92.75%
2025-03-25 14:04:09 - INFO - Performance:
2025-03-25 14:04:09 - INFO - -- True Positives: 171
2025-03-25 14:04:09 - INFO - -- False Positives: 270
2025-03-25 14:04:09 - INFO - -- Loss Contribution: 0.3382
2025-03-25 14:04:09 - INFO - Average Confidence:
2025-03-25 14:04:09 - INFO - -- Correct Predictions: 15.49%
2025-03-25 14:04:09 - INFO - -- Incorrect Predictions: 27.61%
2025-03-25 14:04:09 - INFO - 
Loss Analysis - Cardiomegaly
2025-03-25 14:04:09 - INFO - Confidence Distribution:
2025-03-25 14:04:09 - INFO - -- High (>0.9): 1.14%
2025-03-25 14:04:09 - INFO - -- Medium (0.6-0.9): 3.92%
2025-03-25 14:04:09 - INFO - -- Uncertain (0.4-0.6): 3.46%
2025-03-25 14:04

3378/3378 ━━━━━━━━━━━━━━━━━━━━ 2667s 773ms/step - accuracy: 0.9075 - auc: 0.7740 - f1_score: 0.1197 - loss: 0.1804 - precision: 0.4829 - recall: 0.0867 - val_accuracy: 0.9087 - val_auc: 0.7294 - val_f1_score: 0.1004 - val_loss: 0.2727 - val_precision: 0.3534 - val_recall: 0.0810 - learning_rate: 1.0000e-04


2025-03-25 14:04:24 - INFO - 
=== Training Completed! ===

2025-03-25 14:04:24 - INFO - Final Metrics: accuracy: 0.9077 - auc: 0.7759 - f1_score: 0.1205 - loss: 0.1791 - precision: 0.4858 - recall: 0.0874 - val_accuracy: 0.9087 - val_auc: 0.7294 - val_f1_score: 0.1004 - val_loss: 0.2727 - val_precision: 0.3534 - val_recall: 0.0810

